# Acquire and Prepare

---

This notebook contains all steps taken in the data acquisition and preparation phases of the data science pipeline for the Zillow clustering project. This notebook does rely on helper files so if you want to run the code blocks in this notebook ensure that you have all the helper files in the same directory.

---

## The Required Imports

As stated before this notebook relies on some helper files which are imported below. This notebook also relies on numpy, pandas, matplotlib, seaborn, and sklearn.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from acquire import AcquireZillow
from prepare import *

---

## Data Acquisition

Here we will acquire the data from the zillow database hosted at data.codeup.com. We are acquiring properties with transaction dates in 2017. Additionally, for any properties with more than one transaction in 2017 we are only acquiring the most recent transaction. Finally, we are acquiring only single unit/single family homes.

In [2]:
# All the code needed to acquire the data is encapsulated in the acquire.py file using the following parameters.
#
# Cache file: zillow.csv
# Database name: zillow
# Hostname: data.codeup.com
# 
# SQL Query:
#
#         SELECT
#             properties_2017.*,
#             logerror,
#             transactiondate,
#             typeconstructiondesc,
#             airconditioningdesc,
#             architecturalstyledesc,
#             buildingclassdesc,
#             propertylandusedesc,
#             storydesc,
#             heatingorsystemdesc
#         FROM properties_2017
#         JOIN predictions_2017 ON properties_2017.parcelid = predictions_2017.parcelid
#             AND predictions_2017.transactiondate LIKE '2017%%'
#         LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
#         LEFT JOIN airconditioningtype USING (airconditioningtypeid)
#         LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
#         LEFT JOIN buildingclasstype USING (buildingclasstypeid)
#         LEFT JOIN propertylandusetype USING (propertylandusetypeid)
#         LEFT JOIN storytype USING (storytypeid)
#         LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)

#         JOIN (
#             SELECT
#                 parcelid,
#                 MAX(transactiondate) AS date
#             FROM predictions_2017
#             GROUP BY parcelid
#         ) AS max_dates ON properties_2017.parcelid = max_dates.parcelid
#             AND predictions_2017.transactiondate = max_dates.date

#         WHERE latitude IS NOT NULL AND longitude IS NOT NULL;

raw_zillow_data = AcquireZillow().get_data()

/var/folders/95/v52kk6ys7s14hmmc3xm7b1xr0000gn/T/ipykernel_9984/2514580893.py:42: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  raw_zillow_data = AcquireZillow().get_data()


In [3]:
# How many observations do we have? 77380
raw_zillow_data.shape

(77380, 72)

In [4]:
# Let's see what columns we can drop before moving on to preparation.
raw_zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 72 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   heatingorsystemtypeid         49439 non-null  float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77380 non-null  float64
 3   buildingclasstypeid           15 non-null     float64
 4   architecturalstyletypeid      206 non-null    float64
 5   airconditioningtypeid         24953 non-null  float64
 6   typeconstructiontypeid        222 non-null    float64
 7   id                            77380 non-null  int64  
 8   parcelid                      77380 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77380 non-null  float64
 11  bedroomcnt                    77380 non-null  float64
 12  buildingqualitytypeid         49671 non-null  float64
 13  c

In [5]:
# Let's drop all the foreign key id columns, those will be useless to us.
drop_columns = [
    'heatingorsystemtypeid',
    'storytypeid',
    'propertylandusetypeid',
    'buildingclasstypeid',
    'architecturalstyletypeid',
    'airconditioningtypeid',
    'typeconstructiontypeid',
    'id',
    'parcelid'
]

raw_zillow_data = raw_zillow_data.drop(columns = drop_columns)
raw_zillow_data.shape

(77380, 63)

In [6]:
# Let's make sure it worked.
raw_zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 63 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   basementsqft                  50 non-null     float64
 1   bathroomcnt                   77380 non-null  float64
 2   bedroomcnt                    77380 non-null  float64
 3   buildingqualitytypeid         49671 non-null  float64
 4   calculatedbathnbr             76771 non-null  float64
 5   decktypeid                    614 non-null    float64
 6   finishedfloor1squarefeet      6023 non-null   float64
 7   calculatedfinishedsquarefeet  77184 non-null  float64
 8   finishedsquarefeet12          73748 non-null  float64
 9   finishedsquarefeet13          41 non-null     float64
 10  finishedsquarefeet15          3009 non-null   float64
 11  finishedsquarefeet50          6023 non-null   float64
 12  finishedsquarefeet6           386 non-null    float64
 13  f

Everything looks good. Let's move on to preparation.

---

## Data Preparation

Now we will begin preparing the data. The goal here is to handle all missing values and split the data to prevent data leakage.

---

### First Run Through Pipeline

Here the preparation steps taken in the first run through the pipeline to achieve an MVP will be outlined. We will keep things simple and simply try to drop all the nulls without losing too many rows.

#### Drop Nulls

In [7]:
# Let's set the max rows to display since there will be alot.
pd.set_option('display.max_rows', 70)

# Let's also create a copy of our raw data, since we will be making multiple passes through the pipeline.
zillow = raw_zillow_data.copy()

In [8]:
# Let's see how many null values we have.
summarize_column_nulls(zillow)

,rows_missing,percent_missing
basementsqft,77330,0.999354
bathroomcnt,0,0.000000
bedroomcnt,0,0.000000
buildingqualitytypeid,27709,0.358090
calculatedbathnbr,609,0.007870
decktypeid,76766,0.992065
finishedfloor1squarefeet,71357,0.922163
calculatedfinishedsquarefeet,196,0.002533
finishedsquarefeet12,3632,0.046937
finishedsquarefeet13,77339,0.999470


There are some columns with a large number of missing values. We'll drop those to begin with.

In [9]:
# We'll require 80 percent of the column not having missing values for it to not be dropped.
zillow = drop_missing_values(zillow, prop_required_column = 0.8)

In [10]:
# Let's see what we have now.
zillow.shape

(77380, 30)

In [11]:
# Let's check the null counts again.
summarize_column_nulls(zillow)

,rows_missing,percent_missing
bathroomcnt,0,0.000000
bedroomcnt,0,0.000000
calculatedbathnbr,609,0.007870
calculatedfinishedsquarefeet,196,0.002533
finishedsquarefeet12,3632,0.046937
fips,0,0.000000
fullbathcnt,609,0.007870
latitude,0,0.000000
longitude,0,0.000000
lotsizesquarefeet,8239,0.106475


There aren't too many missing values remaining so we will try removing all rows with missing values.

In [12]:
# We're going to remove any row with a missing value.
zillow = drop_missing_values(zillow, prop_required_row = 1)

In [13]:
# Let's see what we have.
zillow.shape

(64064, 30)

In [14]:
# Let's make sure it worked.
summarize_column_nulls(zillow).sum()

rows_missing       0.0
percent_missing    0.0
dtype: float64

We're all good. In a second pass we can try handling the missing values in a more sophisticated manner, but before we move on to exploration let's encapsulate all these steps into a function.

#### Put it All in a Function

In [17]:
def prepare(df):
    df_copy = df.copy()
    return drop_missing_values(df_copy, prop_required_column = 0.8, prop_required_row = 1)

In [18]:
# Let's make sure it works.
prepare(raw_zillow_data).shape

(64064, 30)

That matches what we had before. Everything looks good.

---

### Second Run Through Pipeline

something here